# BDD Exercises

## Installing and import dependencies

In [ ]:
! pip install matplotlib
! pip install networkx
! pip install omega

In [2]:
from omega.symbolic.fol import Context
import networkx as nx

## Tea party

You want to invite your 5 closest friends to a tea party:
Alex, Bea, Casper, Daniel, and Emily. However, you have to take into account
certain conditions the guests must meet. Because you are prepared, you also
formulated these conditions. Let a, b, c, d, e Boolean variables signify if a certain
person attends the party or not. The conditions are as follows:

- C1: Alex and Bea are dating, they only come together.
- C2: Casper and Bea used to date. So if Bea is present, Casper does not attend.
- C3: If Alex attends, he wants to make sure that Daniel or Emily is present.
- C4: You agreed with Casper that he attends if and only if Daniel and Emily both attend.

What are the possible guest combinations?

IBEN input:
```
vars a b c d e
show ( a = b ) & (b => !c) & ( a => ( d | e ) ) & ( c = (d & e) )
```

In [ ]:
ctx = Context()
ctx.declare(a='bool',b='bool',c='bool',d='bool',e='bool')

C1 = ctx.to_bdd("a = b")
C2 = ctx.to_bdd("b => ~c")
C3 = ctx.to_bdd("a => (d | e)")
C4 = ctx.to_bdd("c = (d & e)")

R = C1 & C2 & C3 & C4

print("Possible guest combinations:")
for d in ctx.pick_iter(R):
    print([ person for person,attends in d.items() if attends ])

## Magic square

Find **_all_** possible ways of filling in an *n x n* table so that:
- all values are a single-digit number
- the sum of each row, column, and the main diagonals are the same number.

In [ ]:
SIZE = 3
SUM = 12 # use None for no restriction

# create the omega context and declare variables
ctx = Context()
ctx.declare(**{f'r{i}c{j}': (0,10) for i in range(SIZE) for j in range(SIZE) })
ctx.declare(**{f'sum': (0,SIZE*10) })

# ensure valid values in cells
bdd = ctx.to_bdd('TRUE')
for i in range(SIZE):
    for j in range(SIZE):
        bdd &= ctx.to_bdd(f'r{i}c{j} < 10')

# restrict the sum
if SUM is not None:
    bdd &= ctx.to_bdd(f'sum = {SUM}')

# rows have the same value
for i in range(SIZE):
    bdd &= ctx.to_bdd('sum = ' + '+'.join([ f'r{i}c{j}' for j in range(SIZE) ]))

# columns have the same value
for j in range(SIZE):
    bdd &= ctx.to_bdd('sum = ' + '+'.join([ f'r{i}c{j}' for i in range(SIZE) ]))

# diagonals have the same value
bdd &= ctx.to_bdd('sum = ' + '+'.join([ f'r{i}c{j}' for i,j in zip(range(SIZE),range(SIZE)) ]))
bdd &= ctx.to_bdd('sum = ' + '+'.join([ f'r{i}c{j}' for i,j in zip(range(SIZE),reversed(range(SIZE))) ]))

# draw magic squares
def draw_square(d):
    print(f"SUM={d['sum']}")
    for i in range(SIZE):
        for j in range(SIZE):
            print(d[f'r{i}c{j}'],end='')
        print()

print('TOTAL NUMBER OF SOLUTIONS:',ctx.count(bdd))
for d in ctx.pick_iter(bdd):
    draw_square(d)
    print()

## Quantifiers

BDDs support both universal and existential quantifiers.
Given _a_, _b_, and _x_ 8-bit values, we can easily check the equivalence of the following expressions:
- a >= b
- \exists x: a=b+x
- \forall x: x>0 => a+x!=b

In [14]:
ctx = Context()
ctx.declare(a=(0,127),b=(0,127),x=(0,127))

bdd_1 = ctx.to_bdd('a >= b')
bdd_2 = ctx.exist(['x'],ctx.to_bdd('a = b + x'))
bdd_3 = ctx.forall(['x'],ctx.to_bdd('x != 0 => a + x != b'))

assert bdd_1 == bdd_2
assert bdd_1 == bdd_3

## Reachability in directed graphs

### Part 1

Given a set of directed edges, collect all *(u,v)* node pairs as a BDD where there is a directed path between *u* and *v*. 

In [ ]:
# First we create a random graph. 
# The seed parameter makes sure, 
# that we can generate the same graph multiple times.

G = nx.erdos_renyi_graph(10,0.15,seed=5555,directed=True)
nx.draw_networkx(G, arrows=True)

In [ ]:
# create an omega context
ctx = Context()
NodeIdInterval = (0,len(G.nodes)-1)
ctx.declare(u=NodeIdInterval,v=NodeIdInterval,w=NodeIdInterval)

# Each node x has a path to itself
HasPath = ctx.to_bdd('FALSE')
for x in G.nodes:
    HasPath |= ctx.to_bdd(f'u={x} & v={x}')

# If there is a direct edge between u and v, then u has a path to v.
for src,dst in G.edges:
    HasPath |= ctx.to_bdd(f'u={src} & v={dst}')

# HasPath(u,w) and HasPath(w,v) => HasPath(u,v)
# Repeate the steps until we reach the minimal fixed-point. 
while True:
    print('Pairs found:',ctx.count(HasPath))
    FirstPart = ctx.let({'v':'w'},HasPath)
    SecondPart = ctx.let({'u':'w'},HasPath)
    NewHasPath = FirstPart & SecondPart
    NewHasPath = ctx.exist(['w'],NewHasPath)
    if HasPath==NewHasPath: break
    HasPath = NewHasPath 

print('Total number of (u,v) pairs:',ctx.count(HasPath))

In [ ]:
# a quick test
for u in G.nodes:
    for v in G.nodes:
        assert (ctx.let({'u':u,'v':v},HasPath)==ctx.true) == nx.has_path(G,u,v)

### Part 2

Using the previously created *HasPath* BDD, provide all *u* nodes that can reach every other node.


In [ ]:
# \forall v: ValidNodeId(v) => HasPath(u,v) 
LuckyNodes = ctx.forall(['v'], ~ctx.to_bdd(f'v<{len(G.nodes)}') | HasPath )
print('Number of dominating nodes:',ctx.count(LuckyNodes))
print(sorted([ d['u'] for d in ctx.pick_iter(LuckyNodes) ]))

In [ ]:
# a quick test
bdd_answer = sorted([ d['u'] for d in ctx.pick_iter(LuckyNodes) ])
nx_answer = [ u for u in G.nodes if all( nx.has_path(G,u,v) for v in G.nodes ) ]
assert bdd_answer == nx_answer